In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
import shutil

In [2]:
# Carregar o modelo treinado
model = tf.keras.models.load_model(r"model/modelo_solda.h5")

# Diretório de novas imagens
input_dir = r"image/classificacao/novas"
train_dir = r"image/classificacao"  # Onde a IA vai mover as imagens classificadas
classes = ["boa", "ruim", "incertas"]

In [3]:
# Caminho para salvar os resultados
resultado_path = "classificacao_resultados.txt"
with open(resultado_path, "w") as f:
    f.write("Imagem,Certeza,Correta\n")

In [4]:
# Função para garantir que as pastas existam
def verificar_pastas():
    for classe in classes:
        pasta = os.path.join(train_dir, classe)
        if not os.path.exists(pasta):
            os.makedirs(pasta)
            print(f"Pasta {classe} criada em {pasta}")
    incerta_pasta = os.path.join(train_dir, "incertas")
    if not os.path.exists(incerta_pasta):
        os.makedirs(incerta_pasta)
        print(f"Pasta 'incertas' criada em {incerta_pasta}")

verificar_pastas()

In [5]:
# Função para emparelhar imagens consecutivas (ex: SNAP0000.jpg + SNAP0001.jpg)
def emparelhar_imagens_consecutivas(origem_dir):
    imagens = sorted([f for f in os.listdir(origem_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
    pares = []
    for i in range(0, len(imagens) - 1, 2):
        img1 = os.path.join(origem_dir, imagens[i])
        img2 = os.path.join(origem_dir, imagens[i + 1])
        pares.append([img1, img2])
    return pares

In [6]:
# Processar e concatenar duas imagens
def processar_imagens(img_path1, img_path2):
    img1 = cv2.imread(img_path1)
    img2 = cv2.imread(img_path2)
    img1 = cv2.resize(img1, (128, 128))
    img2 = cv2.resize(img2, (128, 128))
    img_concatenada = np.concatenate((img1, img2), axis=1)
    img_concatenada = img_concatenada / 255.0
    img_concatenada = np.expand_dims(img_concatenada, axis=0)
    return img_concatenada

In [ ]:
# Classificar os pares de imagens
pares = emparelhar_imagens_consecutivas(input_dir)
for img1_path, img2_path in pares:
    nome_base1 = os.path.basename(img1_path)
    nome_base2 = os.path.basename(img2_path)
    nome_agrupado = f"{nome_base1} + {nome_base2}"

    img_processada = processar_imagens(img1_path, img2_path)
    predicao = model.predict(img_processada)[0]
    classe_predita = np.argmax(predicao)
    classe_nome = classes[classe_predita]
    certeza = predicao[classe_predita] * 100

    print(f"Par {nome_agrupado} classificado como: {classe_nome} ({certeza:.2f}% de certeza)")

    resposta = input("A classificação está correta? (s/n): ").strip().lower()
    correta = 1 if resposta == "s" else 0

    with open(resultado_path, "a") as f:
        f.write(f"{nome_agrupado},{certeza:.2f},{correta}\n")

    if resposta == "n":
        nova_classe = input("Digite a classe correta (boa/ruim/incerta): ").strip().lower()
        if nova_classe in classes:
            destino = os.path.join(train_dir, nova_classe)
            shutil.move(img1_path, os.path.join(destino, nome_base1))
            shutil.move(img2_path, os.path.join(destino, nome_base2))
            print(f"Imagens movidas para: {destino}")
        else:
            print("Classe inválida! As imagens não foram movidas.")
    else:
        destino = os.path.join(train_dir, classe_nome)
        shutil.move(img1_path, os.path.join(destino, nome_base1))
        shutil.move(img2_path, os.path.join(destino, nome_base2))
        print(f"Imagens movidas para: {destino}")

print("Classificação finalizada!")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step
Par SNAP0000.jpg + SNAP0001.jpg classificado como: boa (100.00% de certeza)
Imagens movidas para: image/classificacao\boa
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Par SNAP0002.jpg + SNAP0003.jpg classificado como: boa (100.00% de certeza)
Imagens movidas para: image/classificacao\boa
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Par SNAP0004.jpg + SNAP0005.jpg classificado como: boa (100.00% de certeza)
Imagens movidas para: image/classificacao\boa
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Par SNAP0006.jpg + SNAP0007.jpg classificado como: boa (100.00% de certeza)
Imagens movidas para: image/classificacao\boa
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Par SNAP0008.jpg + SNAP0009.jpg classificado como: boa (100.00% de certeza)
Imagens movidas para: image/classificacao\boa
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Par SNAP0010.jpg + SNAP0011.jpg classificado como: boa (100.00% de certeza)
Imagens movidas para: image/classificacao\boa
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
P